This notebook is for:

- generating parameters for all possible distributions with GARCH.
- Whole data will be used i.e. it won't be divided into test, train data
- final output will be GARCH parameters for **GED**, **norm**, **t**, **skew-t** distributions

In [1]:
!pip install arch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.3/985.3 kB 31.8 MB/s eta 0:00:00


In [2]:
import pandas as pd
from arch import arch_model
import numpy as np


# Load clean dat

In [3]:
# importing clean data
data=pd.read_feather('/content/clean_data_cols_price_lr_slr_sslr_rv_srv.feather')

# Keep only useful columns

In [4]:
main_data=data[['scaled_log_return','scaled_realised_volatility_of_5']]

# Preparing data

## Setting value for using past data

In [5]:
past_data_size=23

## Function to convert data in **window** format

In [8]:
def convert_in_windows(scaled_log_returns, scaled_realised_volatility, window_size=23):
  # create an empty numpy array
  windowed_scaled_log_returns=np.array([])


  for i in range(window_size,len(scaled_log_returns)+1): # here + 1 because we want it to run when i = len(scaled_log_returns) because of "scaled_log_returns.iloc[i-window_size:i]"
    for_appending = scaled_log_returns.iloc[i-window_size:i].values.reshape(1,-1) # (0:22)
    if windowed_scaled_log_returns.shape[0] == 0:
      windowed_scaled_log_returns = for_appending
    else:
      windowed_scaled_log_returns = np.append(windowed_scaled_log_returns,for_appending,axis=0)

  target_scaled_realised_volatility = scaled_realised_volatility.iloc[window_size:].values.reshape(-1,1)

  return windowed_scaled_log_returns, target_scaled_realised_volatility

In [9]:
windowed_data,Y=convert_in_windows(main_data['scaled_log_return'],main_data['scaled_realised_volatility_of_5'])

In [10]:
windowed_data.shape

(6288, 23)

# Function to get **GARCH** parameters

In [ ]:
def get_params_for_GARCH(windowed_data,GARCH_args={ 'mean':'Constant','vol':'GARCH','p':1,'q':1,'dist':'ged'}):

  from arch import arch_model

  params_from_fits=np.array([])

  for window in windowed_data:
    garch_1_1=arch_model(window,mean=GARCH_args['mean'],vol=GARCH_args['vol'],p=GARCH_args['p'],q=GARCH_args['q'],dist=GARCH_args['dist'])
    garch_1_1_fit=garch_1_1.fit(disp=False)
    to_append=garch_1_1_fit.params.loc[['alpha[1]','beta[1]']].values.reshape(1,-1)

    if params_from_fits.shape[0]==0:
      params_from_fits=to_append
    else:
      params_from_fits=np.append(params_from_fits,to_append,axis=0)
  return params_from_fits

# Getting parameters of models

## GARCH parameters for each error distribution

### Generalised Error Distribution

In [ ]:
params_GARCH_ged=get_params_for_GARCH(windowed_data,GARCH_args={ 'mean':'Constant','vol':'GARCH','p':1,'q':1,'dist':'ged'})

/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encountered in power
  lls -= 0.5 * abs(resids / (sqrt(sigma2) * c)) ** nu
/usr/local/lib/python3.11/dist-packages/arch/univariate/distribution.py:1192: RuntimeWarning: overflow encount

### Normal distribution

In [ ]:
params_GARCH_norm=get_params_for_GARCH(windowed_data,GARCH_args={ 'mean':'Constant','vol':'GARCH','p':1,'q':1,'dist':'normal'})

/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09975. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09863. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y

### t distribution

In [ ]:
params_GARCH_t=get_params_for_GARCH(windowed_data,GARCH_args={ 'mean':'Constant','vol':'GARCH','p':1,'q':1,'dist':'t'})

/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09975. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.09863. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 10 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:309: DataScaleWarning: y

### Skewed t distribution

In [ ]:
params_GARCH_skewt=get_params_for_GARCH(windowed_data,GARCH_args={ 'mean':'Constant','vol':'GARCH','p':1,'q':1,'dist':'skewt'})

/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The optimizer returned code 9. The message is:
Iteration limit reached
See scipy.optimize.fmin_slsqp for code meaning.

  warnings.warn(
/usr/local/lib/python3.11/dist-packages/arch/univariate/base.py:768: ConvergenceWarning: The opt

In [ ]:
params_GARCH_ged.shape, params_GARCH_norm.shape, params_GARCH_t.shape, params_GARCH_skewt.shape

((6288, 2), (6288, 2), (6288, 2), (6288, 2))

# Saving GARCH parameters for each distribution

## GED

In [ ]:
params_GARCH_ged_df  = pd.DataFrame(params_GARCH_ged)
params_GARCH_ged_df.to_feather("params_GARCH_ged.feather")

## Normal

In [ ]:
params_GARCH_norm_df  = pd.DataFrame(params_GARCH_norm)
params_GARCH_norm_df.to_feather("params_GARCH_norm.feather")

## t

In [ ]:
params_GARCH_t_df  = pd.DataFrame(params_GARCH_t)
params_GARCH_t_df.to_feather("params_GARCH_t.feather")

## Skew t

In [ ]:
params_GARCH_skewt_df  = pd.DataFrame(params_GARCH_skewt)
params_GARCH_skewt_df.to_feather("params_GARCH_skewt.feather")

---

In [ ]:
temp_ged_params=pd.read_feather('/content/params_GARCH_ged.feather')